In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

In [2]:
Stocks = pd.read_excel('data\Остатки загрузка АЛМ.xlsx')
Stocks = Stocks.rename({'Дата':'ДАТА', 'Артикул':'АРТИКУЛ', 'Цена':'ЦЕНА','Кон ост':'ОСТАТОК','Расход':'РАСХОД'}, axis=1)
Stocks['ДАТА'] = Stocks['ДАТА'].apply(lambda x: x[:-8])
Stocks['ДАТА'] = pd.to_datetime(Stocks['ДАТА'] , format='%d.%m.%Y')
curr_date = str(Stocks['ДАТА'].dt.date.max())


In [3]:
alm = pd.read_excel('output\Потребность АЛМ '+curr_date+'.xlsx')
ast = pd.read_excel('output\Потребность АСТ '+curr_date+'.xlsx')
smt = pd.read_excel('output\Потребность ШМТ '+curr_date+'.xlsx')

alm['ДУБЛИКАТЫ'] = alm['ДУБЛИКАТЫ'].apply(lambda x: '' if x is np.nan else x)
alm['united_code'] = alm['ДУБЛИКАТЫ'] +', '+alm['АРТИКУЛ'].astype('str')
alm['united_code'] = alm['united_code'].apply( lambda x: re.sub(r'[\[\],]','', x))
alm['united_name'] = alm['ТОВАР'] +' '+ alm['ЕД ИЗМ'] +' '+ alm['ПОСТАВЩИК']
alm['region'] = 'АЛМ'

ast['ДУБЛИКАТЫ'] = ast['ДУБЛИКАТЫ'].apply(lambda x: '' if x is np.nan else x)
ast['united_code'] = ast['ДУБЛИКАТЫ'] +', '+ast['АРТИКУЛ'].astype('str')
ast['united_code'] = ast['united_code'].apply( lambda x: re.sub(r'[\[\],]','', x))
ast['united_name'] = ast['ТОВАР'] +' '+ ast['ЕД ИЗМ'] +' '+ ast['ПОСТАВЩИК']
ast['region'] = 'АСТ'

smt['ДУБЛИКАТЫ'] = smt['ДУБЛИКАТЫ'].apply(lambda x: '' if x is np.nan else x)
smt['united_code'] = smt['ДУБЛИКАТЫ'] +', '+smt['АРТИКУЛ'].astype('str')
smt['united_code'] = smt['united_code'].apply( lambda x: re.sub(r'[\[\],]','', x))
smt['united_name'] = smt['ТОВАР'] +' '+ smt['ЕД ИЗМ'] +' '+ smt['ПОСТАВЩИК']
smt['region'] = 'ШМТ'

Определяем пересечения

In [4]:
total = pd.concat([alm,ast,smt], axis=0)
total_united_codes = total.groupby(['united_name', 'ПОСТАВЩИК'])['united_code'].unique().reset_index()


total_united_codes['united_code'] = total_united_codes['united_code'].astype('str').apply( lambda x: re.sub(r'[\[\],]','', x).split()).astype('str')

total_united_codes['united_code'] = total_united_codes['united_code'].apply( lambda x: set(re.sub(r'[\[\],\']','', x).split()) ^ {'""',})


total = pd.pivot_table(data=total, columns='region', index=['united_name','ТОВАР','ПОСТАВЩИК'], values=['order','curr_overstok'], aggfunc='sum').fillna(0).reset_index()


def set_un_code(x):
    return total_united_codes[total_united_codes['united_name'] == x]['united_code'].iloc[0]

total['united_code'] = total['united_name'].apply(set_un_code)

def count_needs(from_, to_, un_name):
    return min(total[total['united_name']==un_name][('curr_overstok', from_)].iloc[0] , total[total['united_name']==un_name][('order', to_)].iloc[0])


total['ШМТ-АЛМ'] = total['united_name'].apply(lambda x: count_needs('ШМТ','АЛМ',x))
total[('order','АЛМ')] = total[('order','АЛМ')] - total['ШМТ-АЛМ'] 
total[('curr_overstok','ШМТ')] = total[('curr_overstok','ШМТ')] - total['ШМТ-АЛМ'] 

total['АСТ-АЛМ'] = total['united_name'].apply(lambda x: count_needs('АСТ','АЛМ',x))
total[('order','АЛМ')] = total[('order','АЛМ')] - total['АСТ-АЛМ'] 
total[('curr_overstok','АСТ')] = total[('curr_overstok','АСТ')] - total['АСТ-АЛМ'] 

total['АЛМ-ШМТ'] = total['united_name'].apply(lambda x: count_needs('АЛМ','ШМТ',x))
total[('order','ШМТ')] = total[('order','ШМТ')] - total['АЛМ-ШМТ'] 
total[('curr_overstok','АЛМ')] = total[('curr_overstok','АЛМ')] - total['АЛМ-ШМТ'] 

total['АЛМ-АСТ'] = total['united_name'].apply(lambda x: count_needs('АЛМ','АСТ',x))
total[('order','АСТ')] = total[('order','АСТ')] - total['АЛМ-АСТ'] 
total[('curr_overstok','АЛМ')] = total[('curr_overstok','АЛМ')] - total['АЛМ-АСТ'] 

total['sum_movement'] = total['ШМТ-АЛМ'] + total['АСТ-АЛМ'] + total['АЛМ-ШМТ'] + total['АЛМ-АСТ']

total = total[total['sum_movement']>0].drop(['sum_movement'], axis=1)
total.to_excel('output\Перемещения '+curr_date+'.xlsx')


C:\Users\killr\AppData\Local\Temp\ipykernel_12200\3354644002.py:40: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  total = total[total['sum_movement']>0].drop(['sum_movement'], axis=1)
